<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/ALMA_7B_Ja_V2_Free_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[ALMA-7B-Ja-V2](https://huggingface.co/webbigdata/ALMA-7B-Ja-V2)は日英、英日翻訳モデルです。  
[ALMA-7B-Ja-V2](https://huggingface.co/webbigdata/ALMA-7B-Ja-V2) is a Japanese-English and English-Japanese translation model.  


上段メニューの「ランタイム」→「すべてのセルを実行」 で実行できます。  
It can be executed by selecting "Runtime" -> "Run All Cells" from the upper menu.  


速度が遅く感じた場合は量子化版の[ALMA-7B-Ja-V2-GPTQ-Ja-En](https://huggingface.co/webbigdata/ALMA-7B-Ja-V2-GPTQ-Ja-En)を試してみてください  

If you feel the speed is slow, try the quantized version [ALMA-7B-Ja-V2-GPTQ-Ja-En](https://huggingface.co/webbigdata/ALMA-7B-Ja-V2-GPTQ-Ja-En)  


もし、以下のようなエラーが発生した場合は  
if you enconter error below.  

```RuntimeError: probability tensor contains either `inf`, `nan` or element < 0  ```

これはメモリ不足を意味します。num_beamsかtoken sizeか翻訳文の長さを減らしてください。  
It's mean your memory is not enough. decrease your num_beams or token size or translation text length.  


In [1]:
%%bash
pip install huggingface_hub==0.17.3
pip install dataclasses
pip install transformers==4.34.0
pip install accelerate==0.23.0
pip install sentencepiece

In [2]:
import torch
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer

# 0 is your 1st GPU memory. Please adjust to match your actual GPU memory.
max_memory_mapping = { 0 : "12GB", "cpu": "10GIB"}

model = AutoModelForCausalLM.from_pretrained("webbigdata/ALMA-7B-Ja-V2", torch_dtype=torch.bfloat16, device_map="auto", max_memory=max_memory_mapping)
tokenizer = LlamaTokenizer.from_pretrained("webbigdata/ALMA-7B-Ja-V2")

prompt1="Translate this from Japanese to English:\nJapanese: 海賊王におれはなる! 何が嫌いかより何が好きかで自分を語れよ！\nEnglish:"
input_ids = tokenizer(prompt1, return_tensors="pt", padding=True, max_length=200, truncation=True).input_ids.cuda()

# Translation
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=3, max_new_tokens=200, do_sample=True, temperature=0.6, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

prompt2="Translate this from English to Japanese:\nEnglish: May the force be with you. This is the way. I have spoken!\nJapanese:"
input_ids = tokenizer(prompt2, return_tensors="pt", padding=True, max_length=200, truncation=True).input_ids.cuda()
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=3, max_new_tokens=200, do_sample=True, temperature=0.6, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Translate this from Japanese to English:\nJapanese: 海賊王におれはなる! 何が嫌いかより何が好きかで自分を語れよ！\nEnglish:I want to be a pirate king! Speak about what you like more than what you dislike!']
['Translate this from English to Japanese:\nEnglish: May the force be with you. This is the way. I have spoken!\nJapanese:力を合わせよ。これが正しい方法です。私は話しました！']
